___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />



# <font color= paletvioletred > Laboratorio 4 - Analisis Fundamental</font>




### Integrantes: 
- <Strong> Chavez Jacobo Diana Teresa </Strong> 


- <Strong> Calderón Erape Jazmín </Strong>

#### 14 Abril 2019

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

In [1]:
# -- ------------------------------------------------------------- Importar modulos a utilizar -- #
# -- ------------------------------------------------------------- --------------------------- -- #

import ta as ta
import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import plotly.graph_objs as go
import plotly as py

In [2]:
# -- -------------------------------------------------------------- Parametros para Data Frame -- #
# -- ------------------------------------------------------------------- --------------------- -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [3]:
# -- ------------------------------------------------------------------- Parametros para OANDA -- #
# -- ------------------------------------------------------------------- --------------------- -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

#A1_OA_Ak = '2' + '6b62ddbe404c61a1cc9da0ed8395945-52d44ef76c42b62460581783bba6c8e' + 'a'
A1_OA_Ak =  "6ff" + "6703d22fd972709992df543492afe" + "-f954a9914f9b39f72fba177a23eb243" + "b" #mi api
#F1 = "2019-03-01T00:00:00Z"
#F2 = "2017-03-01T10:00:00Z"

In [4]:
# -- -------------------------------------------------------------------- Estructuras de datos -- #
# -- ------------------------------------------------------------------- --------------------- -- #

# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [5]:
# -- ---------------------------------------------------------------- Inicializar API de OANDA -- #
# -- ---------------------------------------------------------------- ------------------------ -- #

api = API(access_token=A1_OA_Ak)

In [6]:
# -- ---------------------------------------------------------------- Función de fechas para descargar precios -- #
# -- ---------------------------------------------------------------- ------------------------ -- -------- -----#
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result



In [7]:
# -- ---------------------------------------------------------------- vetor de fechas por periodos de 15 dias--- #
# -- ---------------------------------------------------------------- ------------------------ -- -------- -----#

start_date = date(2017, 3, 1)
end_date = date(2019, 3, 1)
fechas = date_range(start_date, end_date, 15, 'days')

F1 = fechas[0].strftime('%Y-%m-%dT%H:%M:%S')
F2 = fechas[1].strftime('%Y-%m-%dT%H:%M:%S')

In [8]:
# -- -------------------------------------------------------------- Obtener precios historicos -- #
# -- -------------------------------------------------------------- -------------------------- -- #

params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [9]:
lista = []
for i in range(len(A1_Hist['candles'])-1):
        lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lista)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

In [10]:
pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-03-01 05:00:00,20.11632,20.11746,20.11432,20.11738
1,2017-03-01 05:05:00,20.11738,20.11790,20.11592,20.11745
2,2017-03-01 05:10:00,20.11742,20.11744,20.11602,20.11602
3,2017-03-01 05:15:00,20.11578,20.11688,20.11478,20.11658
4,2017-03-01 05:20:00,20.11658,20.11839,20.11626,20.11654
...,...,...,...,...,...
3160,2017-03-16 03:30:00,19.21670,19.22206,19.21649,19.22180
3161,2017-03-16 03:35:00,19.22180,19.22197,19.22076,19.22108
3162,2017-03-16 03:40:00,19.22104,19.22468,19.22092,19.22458
3163,2017-03-16 03:45:00,19.22460,19.22538,19.22460,19.22536
